<a href="https://www.kaggle.com/code/mrstorvo/movies-series-recommendation?scriptVersionId=113566679" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-tv-shows-and-movies/credits.csv
/kaggle/input/netflix-tv-shows-and-movies/titles.csv


In [2]:
# Library
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import silhouette_score
from pandas.core.common import random_state

from skimage import io

In [3]:
# Loading DataSet
titles = pd.read_csv('/kaggle/input/netflix-tv-shows-and-movies/titles.csv', sep = ',')
credits = pd.read_csv('/kaggle/input/netflix-tv-shows-and-movies/credits.csv', sep = ',')

In [4]:
# GET INSIDE THE ATTRIBUTES
titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [5]:
# THE DATASET HAS TWO TYPES ATTRACTIONS: MOVIE AND TV SHOW.
# I MADE A CHOICE THAT THE RECOMMENDATION WILL GIVE THE SAME TYPE OF ATTRACTION THAT YOUR SUGGESTION.
# FIRST, WE CREATE DATASETS WITH MOVIES ONLY AND OTHER WITH SERIES ONLY.
# AND WE VERIFY THE NULL DATA OF EACH DATASET.
data_movie = titles.query('type == "MOVIE"')
data_movie = data_movie.reset_index()
data_movie.isnull().sum()


index                      0
id                         0
title                      1
type                       0
description                9
release_year               0
age_certification       2364
runtime                    0
genres                     0
production_countries       0
seasons                 3744
imdb_id                  262
imdb_score               315
imdb_votes               331
tmdb_popularity           62
tmdb_score               195
dtype: int64

In [6]:
data_show = titles.query('type == "SHOW"')
data_show = data_show.reset_index()
data_show.isnull().sum()

index                     0
id                        0
title                     0
type                      0
description               9
release_year              0
age_certification       255
runtime                   0
genres                    0
production_countries      0
seasons                   0
imdb_id                 141
imdb_score              167
imdb_votes              167
tmdb_popularity          29
tmdb_score              116
dtype: int64

In [7]:
# WHEN WE RECOMMEND A MOVIE OR TV SHOW, IS USUAL TO THINK IN MOVIES OR TV SHOW IN THE SAME GENRE.
# SO, THE RECOMMENDATION WILL FOCUS IN THE GENRES OF THE SUGGESTION.
# IMPORTANT: THE ATTRIBUTES ['imbd_score', 'imbd_votes', 'imbd_popularity', 'tmdb_score'] HAVE SOME NULL VALUES
# AND WE FILL WITH THE MEAN OF THE ATTRIBUTE.

data_movie.update(data_movie['imdb_score'].fillna(data_movie['imdb_score'].mean()))
data_movie.update(data_movie['imdb_votes'].fillna(data_movie['imdb_votes'].mean()))
data_movie.update(data_movie['tmdb_popularity'].fillna(data_movie['tmdb_popularity'].mean()))
data_movie.update(data_movie['tmdb_score'].fillna(data_movie['tmdb_score'].mean()))
data_movie.isnull().sum()


index                      0
id                         0
title                      1
type                       0
description                9
release_year               0
age_certification       2364
runtime                    0
genres                     0
production_countries       0
seasons                 3744
imdb_id                  262
imdb_score                 0
imdb_votes                 0
tmdb_popularity            0
tmdb_score                 0
dtype: int64

In [8]:
data_show.update(data_show['imdb_score'].fillna(data_show['imdb_score'].mean()))
data_show.update(data_show['imdb_votes'].fillna(data_show['imdb_votes'].mean()))
data_show.update(data_show['tmdb_popularity'].fillna(data_show['tmdb_popularity'].mean()))
data_show.update(data_show['tmdb_score'].fillna(data_show['tmdb_score'].mean()))
data_show.isnull().sum()

index                     0
id                        0
title                     0
type                      0
description               9
release_year              0
age_certification       255
runtime                   0
genres                    0
production_countries      0
seasons                   0
imdb_id                 141
imdb_score                0
imdb_votes                0
tmdb_popularity           0
tmdb_score                0
dtype: int64

In [9]:
# FIRST WE WILL RECOMMEND  MOVIES.

# NOW WE CREATE A DATASET FOR EACH MOVIE GENRE.
# IMPORTANT: SOME ATTRIBUTES DOES NOT HAVE EFFECT IN THE RECOMMENDATION, SO THIS ATTRIBUTES WILL BE EXCLUDED 
#                                               ['index', 'description', 'genres', 'production_countries', 
#                                               'type', 'age_certification', 'runtime', 'seasons','imdb_id','apoio']
# FIRST WILL MAKE A LIST OF EACH GENRE.
# AGAIN WE HAVE A PROBLEM THAT SOME MOVIES DOESN'T HAVE GENRE, SO WE CHOOSE TO CONSIDER THIS MOVIES WITH 'UNKNOWN' GENRE.

k1 = 0

while k1 < (len(data_movie)):
  if data_movie['genres'][k1] == "[]":
    data_movie['genres'][k1] = "[[unknown]]"
  data_movie['genres'][k1] = data_movie['genres'][k1][1:-1]
  data_movie['genres'][k1] = data_movie['genres'][k1].split(", ")
  k1 += 1

k2 = 0
genres_list = []
while k2 < (len(data_movie)):
  k3 = 0
  while k3 < (len(data_movie['genres'][k2])):
    data_movie['genres'][k2][k3] = data_movie['genres'][k2][k3][1:-1]
    genres_list.append(data_movie['genres'][k2][k3])

    k3 += 1

  k2 += 1

genres_list_unique = list(set(genres_list))
genres_list_unique.sort()
genres_list_unique

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['action',
 'animation',
 'comedy',
 'crime',
 'documentation',
 'drama',
 'european',
 'family',
 'fantasy',
 'history',
 'horror',
 'music',
 'reality',
 'romance',
 'scifi',
 'sport',
 'thriller',
 'unknown',
 'war',
 'western']

In [10]:
# NOW WE SAVE THE MOVIE DATASET FOR EACH GENRE

for i in genres_list_unique:
    data_movie_test = data_movie.copy()
    list_apoio = []
    k4 = 0

    while k4 < (len(data_movie)):
        list_apoio.append(data_movie['genres'][k4].count(i))
        k4 += 1

    data_movie_test['apoio'] = list_apoio

    df = data_movie_test.query('apoio == 1')
    df = df.drop(columns = ['index', 'description', 'genres', 'production_countries', 'type', 'age_certification', 'runtime', 'seasons','imdb_id','apoio'])
    df.to_csv(i+"_movie.csv")


In [11]:
# NOW WE MAKE A STUDY FOR EACH GENRE DATASET TO PREPARE TO DO THE RECOMMENDATION.
# TO RECOMMEND A MOVIE WE USE PIPELINE TO NORMALIZE AND PRE-PROCESS THE DATASET.
# THEN WE CAN USE KMEANS TO GROUP THE MOVIES (CLUSTERS) IN EACH GENRE.
# WE CHOOSE THE NUMBER OF CLUSTERS FOR EACHE GENRE USING SILHOUETTE SCORE.
# IN THE END WE WILL SEE HOW GOOD EACH GENRE DATASET IS TO RECOMMEND
# FOR EACH GENRE A SCATTER PLOT WITH THE CLUSTERS

for i in genres_list_unique:
    # organizar tabela do genero (cluster)
    data_genero_1 = pd.read_csv(i+'_movie.csv', sep = ',')
    data_genero = data_genero_1.drop(columns = ['Unnamed: 0', 'id', 'title', 'release_year'], axis = 1)
    
    
    SEED = 1224
    np.random.seed(1224)

    pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=0.8, random_state=SEED))])
    genre_embeddinf_pca = pca_pipeline.fit_transform(data_genero)
    projection = pd.DataFrame(data=genre_embeddinf_pca)

    if len(data_genero) > 100:
      max_clusters = 20
    else:
      max_clusters = 6  

    faixa_n_clusters = [i for i in range (2, max_clusters)]
    valores_silhueta = []

    for k in faixa_n_clusters:
      agrupador = KMeans(n_clusters = k, verbose=False, random_state=SEED)
      labels = agrupador.fit_predict(data_genero)
      media_silhueta = silhouette_score(data_genero, labels)
      valores_silhueta.append(media_silhueta)
    
    best_cluster = valores_silhueta.index(max(valores_silhueta)) + 2

    kmeans_pca = KMeans(n_clusters=best_cluster, verbose=False, random_state=SEED)

    kmeans_pca.fit(projection)

    data_genero_1['cluster_pca'] = kmeans_pca.predict(projection)
    projection['cluster_pca'] = kmeans_pca.predict(projection)
    projection['title'] = data_genero_1['title']
    
    fig = px.scatter(
          projection, x=0, y=1, color = 'cluster_pca', hover_data=[0, 1,'title'])

    print("Genre:" + i)
    print("This genre has %.2f explained variance and accurary %.2f" % ( pca_pipeline[1].explained_variance_.sum(),pca_pipeline[1].explained_variance_ratio_.sum()))
    fig.show()

Genre:action
This genre has 3.65 explained variance and accurary 0.91


Genre:animation
This genre has 3.35 explained variance and accurary 0.83


Genre:comedy
This genre has 3.54 explained variance and accurary 0.89


Genre:crime
This genre has 3.71 explained variance and accurary 0.93


Genre:documentation
This genre has 3.53 explained variance and accurary 0.88


...

Genre:sport
This genre has 3.69 explained variance and accurary 0.91


Genre:thriller
This genre has 3.75 explained variance and accurary 0.94


Genre:unknown
This genre has 2.50 explained variance and accurary 0.81


Genre:war
This genre has 3.63 explained variance and accurary 0.90


Genre:western
This genre has 3.84 explained variance and accurary 0.93


In [12]:
# WE MAKE A RECOMMENDATION USING A SUGGESTION
# TO DO THIS, WE TAKE THE MOVIES IN SAME GENRE OF THE SUGGESTION THAT ARE CLOSER IN THE SCATTER PLOT
# THIS METHOD IS CALLED EUCLIDEAN DISTANCES


nome_filme = 'Se7en'


for i in data_movie[data_movie['title'] == nome_filme]['genres'].values[0]:
    # organizar tabela do genero (cluster)
    data_genero_1 = pd.read_csv(i+'_movie.csv', sep = ',')
    data_genero = data_genero_1.drop(columns = ['Unnamed: 0', 'id', 'title', 'release_year'], axis = 1)
    
    
    SEED = 1224
    np.random.seed(1224)

    pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=0.8, random_state=SEED))])
    genre_embeddinf_pca = pca_pipeline.fit_transform(data_genero)
    projection = pd.DataFrame(data=genre_embeddinf_pca)

    if len(data_genero) > 100:
      max_clusters = 20
    else:
      max_clusters = 6  

    faixa_n_clusters = [i for i in range (2, max_clusters)]
    valores_silhueta = []

    for k in faixa_n_clusters:
      agrupador = KMeans(n_clusters = k, verbose=False, random_state=SEED)
      labels = agrupador.fit_predict(data_genero)
      media_silhueta = silhouette_score(data_genero, labels)
      valores_silhueta.append(media_silhueta)
    
    best_cluster = valores_silhueta.index(max(valores_silhueta)) + 2

    kmeans_pca = KMeans(n_clusters=best_cluster, verbose=False, random_state=SEED)

    kmeans_pca.fit(projection)

    data_genero_1['cluster_pca'] = kmeans_pca.predict(projection)
    projection['cluster_pca'] = kmeans_pca.predict(projection)
    projection['title'] = data_genero_1['title']
    
    cluster = list(data_genero_1[data_genero_1['title']== nome_filme]['cluster_pca'])[0]
    filmes_recomendados = projection[projection['cluster_pca']== cluster][[0,1,'title']]

    x_filme = list(projection[projection['title']== nome_filme][0])[0]
    y_filme = list(projection[projection['title']== nome_filme][1])[0]

    distancias = euclidean_distances(filmes_recomendados[[0,1]], [[x_filme, y_filme]])
    filmes_recomendados['id'] = data_genero_1['id']
    filmes_recomendados['distancias'] = distancias
    recomendada = filmes_recomendados.sort_values('distancias')
    recomendada.to_csv(i+'_recomendada.csv')
    resultado = pd.read_csv(i+'_recomendada.csv', sep = ',')
    print('Filmes recomendados no gênero:'+i)
    print(resultado.head(10))
  
    

Filmes recomendados no gênero:crime
   Unnamed: 0         0         1                   title        id  \
0          19  7.274986  2.847335                   Se7en   tm15897   
1         110  6.865767  3.483617   The Dark Knight Rises   tm39888   
2          43  6.231929  2.064420            The Departed   tm27911   
3          10  6.089848  1.895676              GoodFellas  tm155787   
4          22  5.876439  1.953403  Léon: The Professional  tm116655   
5          42  4.860412  1.907785     Catch Me If You Can  tm111828   
6           0  4.664499  1.247439             Taxi Driver   tm84618   
7          20  4.527140  1.132912                  Snatch   tm25947   
8         166  3.415653  0.698184       The Hateful Eight  tm138905   
9         133  3.270355  0.977989            Nightcrawler  tm151663   

   distancias  
0    0.000000  
1    0.756516  
2    1.304195  
3    1.519937  
4    1.659834  
5    2.590931  
6    3.061750  
7    3.238813  
8    4.417387  
9    4.419448  
Filmes

In [13]:
# FOR THE SERIES, WE PROCCED THE SAME WAY.
k1 = 0

while k1 < (len(data_show)):
  if data_show['genres'][k1] == "[]":
    data_show['genres'][k1] = "[[unknown]]"
  data_show['genres'][k1] = data_show['genres'][k1][1:-1]
  data_show['genres'][k1] = data_show['genres'][k1].split(", ")
  k1 += 1

k2 = 0
genres_list_show = []
while k2 < (len(data_show)):
  k3 = 0
  while k3 < (len(data_show['genres'][k2])):
    data_show['genres'][k2][k3] = data_show['genres'][k2][k3][1:-1]
    genres_list_show.append(data_show['genres'][k2][k3])

    k3 += 1

  k2 += 1

genres_list_show_unique = list(set(genres_list_show))
genres_list_show_unique.sort()
genres_list_show_unique

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



['action',
 'animation',
 'comedy',
 'crime',
 'documentation',
 'drama',
 'european',
 'family',
 'fantasy',
 'history',
 'horror',
 'music',
 'reality',
 'romance',
 'scifi',
 'sport',
 'thriller',
 'unknown',
 'war',
 'western']

In [14]:
for i in genres_list_show_unique:
    data_show_test = data_show.copy()
    list_apoio = []
    k4 = 0

    while k4 < (len(data_show)):
        list_apoio.append(data_show['genres'][k4].count(i))
        k4 += 1

    data_show_test['apoio'] = list_apoio

    df = data_show_test.query('apoio == 1')
    df = df.drop(columns = ['index', 'description', 'genres', 'production_countries', 'type', 'age_certification', 'runtime', 'seasons','imdb_id','apoio'])
    df.to_csv(i+"_show.csv")

In [15]:
for i in genres_list_show_unique:
    # organizar tabela do genero (cluster)
    data_genero_show_1 = pd.read_csv(i+'_show.csv', sep = ',')
    data_genero_show = data_genero_show_1.drop(columns = ['Unnamed: 0', 'id', 'title', 'release_year'], axis = 1)
    
    
    SEED = 1224
    np.random.seed(1224)

    pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=0.8, random_state=SEED))])
    genre_embeddinf_pca = pca_pipeline.fit_transform(data_genero_show)
    projection = pd.DataFrame(data=genre_embeddinf_pca)

    if len(data_genero) > 100:
      max_clusters = 20
    else:
      max_clusters = 6  

    faixa_n_clusters = [i for i in range (2, max_clusters)]
    valores_silhueta = []

    for k in faixa_n_clusters:
      agrupador = KMeans(n_clusters = k, verbose=False, random_state=SEED)
      labels = agrupador.fit_predict(data_genero)
      media_silhueta = silhouette_score(data_genero, labels)
      valores_silhueta.append(media_silhueta)
    
    best_cluster = valores_silhueta.index(max(valores_silhueta)) + 2

    kmeans_pca = KMeans(n_clusters=best_cluster, verbose=False, random_state=SEED)

    kmeans_pca.fit(projection)

    data_genero_show_1['cluster_pca'] = kmeans_pca.predict(projection)
    projection['cluster_pca'] = kmeans_pca.predict(projection)
    projection['title'] = data_genero_show_1['title']
    
    fig = px.scatter(
          projection, x=0, y=1, color = 'cluster_pca', hover_data=[0, 1,'title'])

    print("Genre:" + i)
    print("This genre has %.2f explained variance e accurary %.2f" % ( pca_pipeline[1].explained_variance_.sum(),pca_pipeline[1].explained_variance_ratio_.sum()))
    fig.show()

Genre:action
This genre has 3.55 explained variance e accurary 0.88


Genre:animation
This genre has 3.50 explained variance e accurary 0.87


Genre:comedy
This genre has 3.49 explained variance e accurary 0.87


Genre:crime
This genre has 3.55 explained variance e accurary 0.88


Genre:documentation
This genre has 3.47 explained variance e accurary 0.86


...

Genre:sport
This genre has 3.50 explained variance e accurary 0.86


Genre:thriller
This genre has 3.62 explained variance e accurary 0.90


Genre:unknown
This genre has 3.60 explained variance e accurary 0.86


Genre:war
This genre has 3.63 explained variance e accurary 0.89


Genre:western
This genre has 4.11 explained variance e accurary 0.95


In [16]:
nome_show = "The Mystic River"


for i in data_show[data_show['title'] == nome_show]['genres'].values[0]:
    # organizar tabela do genero (cluster)
    data_genero_show_1 = pd.read_csv(i+'_show.csv', sep = ',')
    data_genero_show = data_genero_show_1.drop(columns = ['Unnamed: 0', 'id', 'title','release_year'], axis = 1)
    
    
    SEED = 1224
    np.random.seed(1224)

    pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=0.8, random_state=SEED))])
    genre_embeddinf_pca = pca_pipeline.fit_transform(data_genero_show)
    projection = pd.DataFrame(data=genre_embeddinf_pca)

    if len(data_genero) > 100:
      max_clusters = 20
    else:
      max_clusters = 6  

    faixa_n_clusters = [i for i in range (2, max_clusters)]
    valores_silhueta = []

    for k in faixa_n_clusters:
      agrupador = KMeans(n_clusters = k, verbose=False, random_state=SEED)
      labels = agrupador.fit_predict(data_genero)
      media_silhueta = silhouette_score(data_genero, labels)
      valores_silhueta.append(media_silhueta)
    
    best_cluster = valores_silhueta.index(max(valores_silhueta)) + 2

    kmeans_pca = KMeans(n_clusters=best_cluster, verbose=False, random_state=SEED)

    kmeans_pca.fit(projection)

    data_genero_show_1['cluster_pca'] = kmeans_pca.predict(projection)
    projection['cluster_pca'] = kmeans_pca.predict(projection)
    projection['title'] = data_genero_show_1['title']
        
    cluster = list(data_genero_show_1[data_genero_show_1['title']== nome_show]['cluster_pca'])[0]
    filmes_recomendados = projection[projection['cluster_pca']== cluster][[0,1,'title']]

    x_filme = list(projection[projection['title']== nome_show][0])[0]
    y_filme = list(projection[projection['title']== nome_show][1])[0]

    distancias = euclidean_distances(filmes_recomendados[[0,1]], [[x_filme, y_filme]])
    filmes_recomendados['id'] = data_genero_show_1['id']
    filmes_recomendados['distancias'] = distancias
    recomendada = filmes_recomendados.sort_values('distancias')
    recomendada.to_csv(i+'_serie_recomendada.csv')
    resultado = pd.read_csv(i+'_serie_recomendada.csv', sep = ',')
    print('Filmes recomendados no gênero:'+i)
    print(resultado.head(10))
    

Filmes recomendados no gênero:horror
   Unnamed: 0         0         1                         title        id  \
0         116 -0.934411  0.374788              The Mystic River  ts296581   
1          90 -0.932143  0.436490                      Vampires  ts315066   
2          95 -0.775410  0.291465  The Forest of Love: Deep Cut  ts237071   
3          29 -0.752449  0.342939                     Crazyhead   ts53155   
4          71 -0.675974  0.467646                  Black Summer   ts89564   
5          81 -0.988931  0.648466                      Chambers   ts81646   
6          42 -0.657287  0.204944                   Creeped Out   ts79660   
7          80 -0.650128  0.195079                        V Wars   ts83107   
8          22 -0.910320  0.716053                       Slasher   ts38718   
9          58 -0.698170  0.127018                Super Monsters   ts76159   

   distancias  
0    0.000000  
1    0.061743  
2    0.179511  
3    0.184728  
4    0.274613  
5    0.279055  
6  

In [17]:
# NOW, WE TAKE THE NAME OF THE LAST MOVIE OR TV SHOW THAT YOU SAW
# AND USE THE RECOMMENDATION!

# THANKS TO TAKE YOUR TIME WITH ME IN THIS PROJECT! PLEASE, SHARE YOUR OPNION, SUGGESTION OR CORRECTION ABOUT IT.